# Playing with Microsoft 365 Defender (M365D) (old MTP) REST API
https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-supported?view=o365-worldwide

Requirements: the APP should have the following 'API permissions':
- Microsoft Threat Protection:
  - AdvancedHunting.Read
  - Incident.Read

**VERY** Important either the global admin allows these list of API permission OR you should give a link to him/her (NOTE to change the app clientid) :
$https://login.microsoftonline.com/common/oauth2/authorize?prompt=consent&client_id=<HERE-THE-CLIENT-ID>&response_type=code&sso_reload=true$


This REST API have 2 exposed APIs. We show bellow the link to each individual exposed API. Each API have multiple methods. 

|#|Topic|Description|Link|
|:-|:-|:-|:-|
|1|Advanced Hunting|Run queries from API.|[link](https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-advanced-hunting?view=o365-worldwide)|
|2|Incident |List and update incidents, along with other practical tasks.|[link](https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-incident?view=o365-worldwide)|


# Required information

In [1]:
appId=''
tenantId=''
appSecret=''

# Required libraries

In [ ]:
import pandas as pd
import json
import urllib.parse

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the token to access via the app

In [ ]:
def m365d_getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://api.security.microsoft.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

In [ ]:
access_token = m365d_getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>========================================================<br>=======================================================<br> ADVANCED HUNTING API</h1>

https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-advanced-hunting?view=o365-worldwide

Note: this API is the same as the of from the MDE/MDATP. Examples of queries [here!](https://github.com/microsoft/Microsoft-365-Defender-Hunting-Queries)

# Function Run (KQL)

In [ ]:
def mde_advancedquery_run(access_token,kql_query):
    url = "https://api.securitycenter.microsoft.com/api/advancedqueries/run"
    req = urllib.request.Request(url, 
                                 headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'},
                                 data=bytes(json.dumps({"Query": kql_query}), encoding="utf-8"))
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['Results'])

# Example of instantiating this function

In [ ]:
kql_query = '''
AlertInfo
|where Timestamp > ago (30d)
'''

In [ ]:
mde_advancedquery_run(access_token,kql_query)

<h1 align='center'>========================================================<br>=======================================================<br>INCIDENT API</h1>

List of methods in the Incident API (https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-incident?view=o365-worldwide

|Method	| Return Type |Description  |
|:- |:-|:-|
|List incidents|	Incident list|	Get a list of incidents.|
|Update incident|	Incident|	Update a specific incident.|

## METHOD List incidents

In [ ]:
def m365d_incidents_listincidents(access_token):
    url = "https://api.security.microsoft.com/api/incidents"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

In [ ]:
df_incidents = m365d_incidents_listincidents(access_token)
df_incidents

<h1 align='center'>...........................................................</h1>

## METHOD Update incident

In [ ]:
def m365d_incidents_updateincidents(access_token,incident_id):
    url = "https://api.security.microsoft.com/api/incidents/"+str(incident_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return jsonResponse

In [ ]:
m365d_incidents_updateincidents(access_token,df_incidents['id'][0])